In [1]:
import pickle
import pandas as pd
import numpy as np
from pyspark.sql import DataFrame
from aeon.classification.distance_based import ProximityTree, ProximityForest
import logging
from pyspark.sql import SparkSession
import os
from pyspark.sql import SparkSession
from data_ingestion import DataIngestion
from preprocessing import Preprocessor
from prediction_manager import PredictionManager
from local_model_manager import LocalModelManager
from evaluation import Evaluator
from utilities import show_compact
import time
import json
from random import sample
from dtaidistance import dtw
import collections
from pprint import pprint
import random
import collections
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.master("local[*]").appName("testingglobal").getOrCreate()
sc = spark.sparkContext

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, ArrayType
import random
import collections

class GlobalProxTree:
    def __init__(self, spark, max_depth=5, min_samples=5):
        """
        Initialize the Time Series Decision Tree
        
        Parameters:
        -----------
        spark : SparkSession
            The Spark session to use
        max_depth : int
            Maximum depth of the tree
        min_samples : int
            Minimum number of samples required to split a node
        """
        self.spark = spark
        self.max_depth = max_depth
        self.min_samples = min_samples
        
        # Define the schema for tagged dataframe
        self.tagged_schema = StructType([
            StructField("node_id",     IntegerType(),           False),
            StructField("time_series", ArrayType(DoubleType()), False),
            StructField("branch_id",   IntegerType(),           False),
            StructField("true_label",  IntegerType(),           False),
            StructField("dist_calc",   DoubleType(),            False),
        ])
        
        # Define the TreeNode structure
        self.TreeNode = collections.namedtuple(
            "TreeNode",
            "node_id parent_id split_on is_leaf prediction children gini_parent".split()
        )
        
        # Initialize the tree with a root node
        self.tree = {
            0: self.TreeNode(
                node_id=0,
                parent_id=None,
                split_on=None,
                is_leaf=False,
                prediction=None,
                children={},
                gini_parent=None
            )
        }
    
    def _convert_to_time_series_format(self, df):
        """
        Convert wide dataframe (with each feature in its own column) to a dataframe 
        with a single array column containing all features
        
        Parameters:
        -----------
        df : Spark DataFrame
            Wide DataFrame with feature columns and label column
            
        Returns:
        --------
        Spark DataFrame
            DataFrame with 'time_series' and 'label' columns
        """
        # Check if 'time_series' column already exists
        if 'time_series' in df.columns:
            print("DataFrame already has 'time_series' column, no conversion needed")
            return df
            
        # Get all column names except 'label'
        feature_cols = [col for col in df.columns if col != 'label']
        
        print(f"Converting {len(feature_cols)} feature columns to 'time_series' array")
        
        # Use array() function to combine columns
        # This is more efficient than a UDF
        ts_df = df.select(
            F.array(*[F.col(c) for c in feature_cols]).alias("time_series"),
            df["label"].cast(IntegerType())  # Ensure label is an integer
        )
        
        # Show sample of converted data
        print("Sample of converted DataFrame:")
        ts_df.show(2, truncate=False)
        
        return ts_df
        
    def fit(self, df):
        """
        Fit the decision tree on the dataframe
        
        Parameters:
        -----------
        df : Spark DataFrame
            DataFrame with feature columns and 'label' column
            
        Returns:
        --------
        self : GlobalProxTree
            The fitted tree
        """
        # First, convert to time_series format if needed
        df = self._convert_to_time_series_format(df)
        
        # Initialize assignment dataframe with all rows at the root node
        assign_df = df.rdd \
            .map(lambda r: (0, r.time_series, r.label)) \
            .toDF(["node_id", "time_series", "true_label"]) \
            
        
        open_nodes = {0}
        
        for depth in range(self.max_depth):
            # If no nodes to expand, stop
            if not open_nodes:
                break
                
            # 1. Sample local exemplars
            exemplars = {}
            # Pull out only the rows at the currently open nodes
            for row in assign_df.filter(F.col("node_id").isin(open_nodes)).collect():
                node, series, lbl = row
                exemplars.setdefault((node, lbl), []).append(series)
            
            # For each (node, class) pick one exemplar at random
            exemplars = {k: random.choice(v) for k, v in exemplars.items()}
            
            # Use a Python function in map instead of a lambda to avoid serializing the class
            def tag_with_exemplars(row, exemplars=exemplars):
                node_id, time_series, true_label = row
                best_branch, best_dist = None, float("inf")
                
                for (node_key, ex_lbl), ex_vec in exemplars.items():
                    if node_key != node_id:
                        continue
                    
                    # Calculate distance (Euclidean distance as fallback if DTW not available)
                    try:
                        import dtw
                        d = dtw.distance(time_series, ex_vec)
                    except ImportError:
                        # Fallback to Euclidean distance
                        d = sum((a - b) ** 2 for a, b in zip(time_series, ex_vec)) ** 0.5
                        
                    if d < best_dist:
                        best_branch, best_dist = ex_lbl, d
                
                if best_branch is None:
                    # Default to first label if no match found (shouldn't happen)
                    print(f"Warning: No exemplar for node {node_id}")
                    best_branch = list(set(ex_lbl for (n, ex_lbl), _ in exemplars.items() if n == node_id))[0]
                    best_dist = float('inf')
                    
                return (node_id, time_series, best_branch, true_label, best_dist)
            
            # 2. Tag every row with nearest exemplar
            tagged = (
                assign_df.rdd
                .map(lambda r: tag_with_exemplars(r))
                .toDF(self.tagged_schema)
                .cache()
            )
            
            # 3. Build histogram & decide splits
            hist = self._build_histogram(tagged, open_nodes, debug=(depth == 0))
            if not hist:
                print(f"No more splits at depth={depth}, stopping.")
                break
                
            # Print debugging information
            print(f"\n=== depth={depth}, open_nodes={open_nodes} ===")
            import pprint
            pprint.pprint(hist)
            
            # Decide splits
            next_open = set()
            for nid in open_nodes:
                if nid in hist:  # Check if the node has any data points
                    children = self._split_node_gini(
                        nid,
                        hist[nid],
                        depth,
                        self.max_depth,
                        self.min_samples
                    )
                    next_open |= children  # Union of new internal nodes
            
            # 4. Push rows down to child node_ids
            assign_df = self._push_rows_down(tagged, open_nodes, next_open)
            
            open_nodes = next_open  # Next layer
            
            # Filter to only rows at open nodes
            assign_df = assign_df.where(F.col("node_id").isin(next_open)).cache()
            
        return self
    
    def _build_histogram(self, tagged_df, open_nodes, debug=False):
        """
        Count rows for every (node_id, branch_id, true_label) among the open nodes.
        
        Parameters:
        -----------
        tagged_df : Spark DataFrame
            DataFrame containing tagged rows
        open_nodes : set
            Set of node IDs to process
        debug : bool
            Whether to print debug information
            
        Returns:
        --------
        dict : Nested dictionary with counts
            { node_id: { branch_id: { true_label: count, ... }, ... }, ... }
        """
        if not open_nodes:
            return {}
            
        # Filter to only the node_ids we're actually expanding
        filtered = tagged_df.where(F.col("node_id").isin(open_nodes))
        counts = (
            filtered
            .groupBy("node_id", "branch_id", "true_label")
            .count()
        )
        
        if debug:
            print(">>> FILTERED ROWS FOR THESE open_nodes:", open_nodes)
            filtered.show(2, truncate=False)
            print(">>> AGGREGATED COUNTS:")
            counts.show(5, truncate=False)
        
        # Build histogram directly from collected rows    
        hist = {}
        for r in counts.collect():
            node_id = r["node_id"]
            branch_id = r["branch_id"]
            true_label = r["true_label"]
            count = r["count"]
            
            if node_id not in hist:
                hist[node_id] = {}
            if branch_id not in hist[node_id]:
                hist[node_id][branch_id] = {}
            hist[node_id][branch_id][true_label] = count
                
        return hist
    
    def _split_node_gini(self, node_id, branches, depth, max_depth, min_samples):
        """
        Decide whether to split a node based on Gini impurity
        
        Parameters:
        -----------
        node_id : int
            ID of the node to potentially split
        branches : dict
            Dictionary of branches for this node
        depth : int
            Current depth in the tree
        max_depth : int
            Maximum allowed depth
        min_samples : int
            Minimum samples required to split
            
        Returns:
        --------
        set : Set of newly created internal nodes
        """
        new_nodes = set()
        
        # Skip if we're at max depth
        if depth >= max_depth - 1:
            self.tree[node_id] = self.tree[node_id]._replace(is_leaf=True)
            
            # Find most common class for prediction
            all_counts = {}
            for branch_id, class_counts in branches.items():
                for label, count in class_counts.items():
                    all_counts[label] = all_counts.get(label, 0) + count
            
            prediction = max(all_counts.items(), key=lambda x: x[1])[0] if all_counts else None
            self.tree[node_id] = self.tree[node_id]._replace(prediction=prediction)
            
            print(f"Node {node_id} is now a leaf with prediction {prediction}")
            return new_nodes
            
        # Create a child node for each branch
        next_node_id = max(self.tree.keys()) + 1
        
        for branch_id in branches:
            # Compute total counts for this branch
            branch_total = sum(count for label, count in branches[branch_id].items())
            
            # Skip branches with too few samples
            if branch_total < min_samples:
                print(f"Branch {branch_id} has only {branch_total} samples, skipping")
                continue
                
            # Create a new child node
            child_id = next_node_id
            next_node_id += 1
            
            # Create the child node
            self.tree[child_id] = self.TreeNode(
                node_id=child_id,
                parent_id=node_id,
                split_on=None,  # No further split yet
                is_leaf=False,  # Will be determined in next iteration
                prediction=None,  # Will be set if it becomes a leaf
                children={},
                gini_parent=None  # Calculate if needed
            )
            
            # Add child to parent's children
            self.tree[node_id].children[branch_id] = child_id
            new_nodes.add(child_id)
            
            print(f"Created child node {child_id} for branch {branch_id}")
            
        return new_nodes
    
    def _push_rows_down(self, tagged_df, old_nodes, new_nodes):
        """
        Push rows down from parent nodes to child nodes
        
        Parameters:
        -----------
        tagged_df : Spark DataFrame
            DataFrame containing tagged rows
        old_nodes : set
            Set of current node IDs
        new_nodes : set
            Set of new node IDs
            
        Returns:
        --------
        Spark DataFrame : Updated assignment DataFrame
        """
        # Create a map from (parent_id, branch_id) to child_id
        parent_branch_to_child = {}
        
        for nid in old_nodes:
            if nid in self.tree:
                for branch_id, child_id in self.tree[nid].children.items():
                    parent_branch_to_child[(nid, branch_id)] = child_id
        
        # Create a closure over the mapping
        mapping = parent_branch_to_child  # Local variable to avoid serializing self
        
        def map_to_child(node_id, branch_id):
            key = (node_id, branch_id)
            if key in mapping:
                return mapping[key]
            return node_id  # Keep at same node if no mapping
        
        # Create a UDF from the closure
        map_to_child_udf = F.udf(map_to_child, IntegerType())
        
        # Apply the mapping
        pushed_df = tagged_df.withColumn(
            "node_id",
            map_to_child_udf(F.col("node_id"), F.col("branch_id"))
        ).select("node_id", "time_series", "true_label").cache()
        
        return pushed_df
    
    def predict(self, df):
        """
        Make predictions using the trained tree
        
        Parameters:
        -----------
        df : Spark DataFrame
            DataFrame with feature columns or 'time_series' column
            
        Returns:
        --------
        Spark DataFrame : DataFrame with predictions
        """
        # First, convert to time_series format if needed
        df = self._convert_to_time_series_format(df)
        
        # Simplified predict implementation
        def traverse_tree(time_series):
            """Traverse the tree for a single time series instance"""
            node_id = 0  # Start at root
            
            while node_id in self.tree and not self.tree[node_id].is_leaf:
                # If we're at a leaf, return its prediction
                if self.tree[node_id].is_leaf:
                    return self.tree[node_id].prediction
                
                # Find distances to all exemplars for this node's children
                min_dist = float('inf')
                best_branch = None
                
                # Simplified based on the exemplar approach
                for branch_id in self.tree[node_id].children:
                    # Using branch_id directly as a proxy for distance
                    # In a full implementation, we would use actual exemplars here
                    if branch_id == time_series[0]:  # Simplified matching
                        best_branch = branch_id
                        break
                
                # Default to first child if no match
                if best_branch is None and self.tree[node_id].children:
                    best_branch = list(self.tree[node_id].children.keys())[0]
                
                # If we have a branch to follow, go there
                if best_branch is not None and best_branch in self.tree[node_id].children:
                    node_id = self.tree[node_id].children[best_branch]
                else:
                    # If we can't go further, use current node's prediction (or default)
                    break
            
            # Return prediction if node exists and is a leaf
            if node_id in self.tree and self.tree[node_id].is_leaf:
                return self.tree[node_id].prediction
            
            # Default prediction (most common class among all data)
            return 1  # Default prediction
        
        # This is a placeholder - in reality you would implement a full traversal
        print("Prediction functionality is simplified - modify for real use")
        return df.withColumn("prediction", F.lit(1))  # Return default prediction
    
    def print_tree(self):
        """
        Print a representation of the tree
        
        Returns:
        --------
        str : String representation of the tree
        """
        def print_node(node_id, depth=0):
            node = self.tree[node_id]
            indent = "  " * depth
            
            if node.is_leaf:
                return f"{indent}Node {node_id}: LEAF, prediction={node.prediction}\n"
            
            result = f"{indent}Node {node_id}: internal\n"
            
            for branch_id, child_id in sorted(node.children.items()):
                result += f"{indent}  Branch {branch_id} -> {child_id}\n"
                result += print_node(child_id, depth + 1)
                
            return result
        
        return print_node(0)  # Start at root

In [4]:
config = {
    "databricks_data_path": "/mnt/2025-team6/fulldataset_ECG5000.csv",
    "local_data_path": "/fulldataset_ECG5000.csv",
    "label_col": "label",
    "data_percentage": 1.0,
    "min_number_iterarations": 2,

    "local_model_config": {
        "test_local_model" : True,
        "num_partitions": 10,  
        "tree_params": {
            "n_splitters": 5,  # Matches ProximityTree default
            "max_depth": None,  
            "min_samples_split": 5,  # From ProximityTree default
            "random_state": 123
            },
        "forest_params": {
            "random_state": 123,
            "n_jobs": -1  # Use all available cores
            }
    },
    "global_model_config": {
        "test_local_model" : False,
        "num_partitions": 10
    }
}

ingestion_config = {
                "data_path":r"D:\repos\BigData-main\BigData-1\fulldataset_ECG5000.csv",
                "data_percentage": config.get("data_percentage", 0.5)
}

In [5]:
ingestion = DataIngestion(spark=spark, config=ingestion_config)
preprocessor = Preprocessor(config=config)

In [6]:
# load + preprocess data
df = ingestion.load_data()
df = preprocessor.run_preprocessing(df)

Data Path: D:\repos\BigData-main\BigData-1\fulldataset_ECG5000.csv
Loading 100.0% of data
Data size: 5000

Repartitioning to 10 workers


+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------

In [8]:
# The original data is in wide format
print("Original DataFrame:")

print(f"DataFrame shape: {len(df.columns)} columns, {df.count()} rows")

# Limit to fewer rows for quicker testing
test_df = df.limit(50)
print(f"Test DataFrame shape: {len(test_df.columns)} columns, {test_df.count()} rows")

Original DataFrame:
DataFrame shape: 141 columns, 5000 rows
Test DataFrame shape: 141 columns, 50 rows


In [9]:
# from pyspark.sql.window import Window
# from pyspark.sql.functions import row_number, rand, col

# # ─── balance to 50 samples per class ─────────────────────────
# w = Window.partitionBy("label").orderBy(rand())
# df = (
#     df
#     .withColumn("rn", row_number().over(w))
#     .filter(col("rn") <= 50)
#     .drop("rn")
# )
# # now df has up to 50 rows for each label

# # ─── inspect! ────────────────────────────────────────────────
# print("Balanced DataFrame:")
# df.printSchema()
# print(f"DataFrame shape: {len(df.columns)} columns, {df.count()} rows")

In [ ]:
# ─── split into train/test ──────────────────────────────────
train_df, test_df = df.randomSplit([0.8, 0.2], seed=1234)

AttributeError: 'DataFrame' object has no attribute 'size'

In [13]:
import pyspark.sql.functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Example usage of the updated GlobalProxTree with the sample data

# First, create the tree with the desired parameters
tree = GlobalProxTree(spark, max_depth=5, min_samples=5)



# Now we can directly fit the tree on the wide DataFrame
# The conversion will happen automatically inside the fit method
try:
    print("\nFitting tree on wide DataFrame...")
    tree.fit(df)
    
    print("\nTree structure:")
    print(tree.print_tree())
except Exception as e:
    print(f"Error while fitting tree: {e}")
    import traceback
    traceback.print_exc()

print("\nTree fitting complete!")

# You can also use the tree for prediction (simplified implementation)
# predictions = tree.predict(new_df)


Fitting tree on wide DataFrame...
Converting 140 feature columns to 'time_series' array
Sample of converted DataFrame:
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Traceback (most recent call last):
  File "C:\Users\Petru\AppData\Local\Temp\ipykernel_30180\4033647616.py", line 15, in <module>
    tree.fit(df)
  File "C:\Users\Petru\AppData\Local\Temp\ipykernel_30180\175154783.py", line 110, in fit
    .toDF(["node_id", "time_series", "true_label"]) \
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Petru\anaconda3\envs\bigdata_env\Lib\site-packages\pyspark\sql\session.py", line 122, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Petru\anaconda3\envs\bigdata_env\Lib\site-packages\pyspark\sql\session.py", line 1443, in createDataFrame
    return self._create_dataframe(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Petru\anaconda3\envs\bigdata_env\Lib\site-packages\pyspark\sql\session.py", line 1483, in _create_dataframe
    rdd, struct = self._createFromRDD(data.map(prepare), schema, samplingRatio)
              

In [11]:


test_df = df 

from pyspark.sql.types import DoubleType

# 2) Run predict() on your held‑out test_df (must have a "label" column):
pred_df = tree.predict(test_df)

# Cast both label and prediction to Double so the evaluator is happy
pred_df = (
    pred_df
    .withColumn("prediction", F.col("prediction").cast(DoubleType()))
    .withColumn("label",      F.col("label")     .cast(DoubleType()))
)

# Show a few predictions vs. truth
pred_df.select("label", "prediction").show(10)

# 3a) Manual accuracy via a “correct” flag + avg:
acc_manual = (
    pred_df
    .withColumn("correct", F.when(F.col("prediction") == F.col("label"), 1).otherwise(0))
    .agg(F.avg("correct").alias("accuracy"))
    .collect()[0]["accuracy"]
)
print(f"Manual accuracy = {acc_manual:.3f}")

# 3b) Or use the built‑in evaluator:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)
acc_evaluator = evaluator.evaluate(pred_df)
print(f"Evaluator accuracy = {acc_evaluator:.3f}")


Converting 140 feature columns to 'time_series' array
Sample of converted DataFrame:
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, DoubleType

def convert_using_udf(df):
    """Alternative method using a UDF"""
    
    feature_cols = [col for col in df.columns if col != 'label']
    
    # Define a UDF to combine values into an array
    @F.udf(returnType=ArrayType(DoubleType()))
    def combine_features(*cols):
        return [float(c) if c is not None else 0.0 for c in cols]
    
    # Apply the UDF to create time_series column
    udf_ts_df = df.select(
        combine_features(*feature_cols).alias("time_series"),
        "label"
    )
    
    return udf_ts_df

# Convert using the UDF method
udf_ts_df = convert_using_udf(df)

print("\nConverted DataFrame (UDF method):")
udf_ts_df.show(5, truncate=False)


Converted DataFrame (UDF method):
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------